# Keras Neural Networks

This is an example of how to use Keras to create a simple neural network. The code for this example was (shamelessly) lifted straight off of the Keras source code [here](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Keras is a wrapper for TensorFlow. It makes it super easy to build and train a wide range of neural networks with very little code. Here, we'll train our neural network to recognize MNIST digits.

In [ ]:
!pip install matplotlib

In [ ]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.optimizers import RMSprop
import numpy as np
import matplotlib.pyplot as plt



# Fully Connected Neural Network

### Define hyperparameters for our network
`batch_size` := the number of images we show per gradient update 

`num_classes` := the number of different classes (or bins) that we can place our data into. Determined by the dataset.

`epochs` := the number of times we go through the dataset during training. 

In [ ]:
batch_size = 128
num_classes = 10 # 10 classes because we have 10 digits (0-9)
epochs = 5

### Data Preparation
Keras has many built-in functions such as `to_categorical` that makes data manipulation very easy. Here we download the mnist dataset (using a built-in Keras function) and then preprocess it:

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# reshape 2d (28x28) image data into 1d vectors (28x28 = 784-d vectors)
x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)

# set type to float, then clamp data to values between 0-1 instead of integers 0 - 255
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class labels into vectors. Ie 3 -> [0,0,0,1,0,0,0,0,0,0]
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Building a network in Keras is incredibly straight forward. Just instantiate a `Sequential` object and then use `.add` to add layers to you network.

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

To train your model, just use use the `.fit` function. If `verbose=1`, then Keras prints out pretty summaries of how your network is doing while training

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def predict(n):
    """Predict the nth test data sample using the model"""
    print("Prediction: " + str(np.round(model.predict(np.expand_dims(x_test[n], 0)), 2)))
    print("Actual: " + str(y_test[n]))
    plt.imshow(x_test[n].reshape(28,28), cmap='Greys')
    plt.show()
    
predict(2354)

# Convolutional Neural Network

### Define hyperparameters for our network
`batch_size` := the number of images we show per gradient update 

`num_classes` := the number of different classes (or bins) that we can place our data into. Determined by the dataset.

`epochs` := the number of times we go through the dataset during training. 

In [ ]:
batch_size = 128
num_classes = 10 # 10 classes because we have 10 digits (0-9)
epochs = 5

### Data Prep
This time, we don't need to reshape our data because we're using a CNN. The CNN performs _better_ with the 2D data because it preserves spatial information. However, we still want to clamp our data between 0 and 255 

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# set type to float, then clamp data to values between 0-1 instead of integers 0 - 255
x_train = x_train.reshape(list(x_train.shape) + [1])
x_test = x_test.reshape(list(x_test.shape) + [1])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class labels into vectors. Ie 3 -> [0,0,0,1,0,0,0,0,0,0]
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

* input layer : 784 nodes (MNIST images size)
* first convolution layer : 5x5x32
* first max-pooling layer
* second convolution layer : 5x5x64
* second max-pooling layer
* third fully-connected layer : 1024 nodes
* output layer : 10 nodes (number of class for MNIST)

In [ ]:
model = None

In [ ]:
model = Sequential()
model.add(Conv2D(32, (5,5), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (5,5), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])